In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma

file_path = './data/Be_Good.pdf'

loader = PyPDFLoader(file_path=file_path)

docs = loader.load()

print(len(docs))
print(docs[0].page_content[:100])

11
Be Good - Essay by Paul Graham
Be Good
Be good
April 2008(This essay is derived from a talk at the 2


### **RAG**

* We will use the vector database (aa vector store) Chroma DB
* Using a text splitter, we will split the loaded PDF into smaller documents that can more easily fit into an LLM context window, then load them in a vector store
* We can then create a retriever from the vector sotre for use in our RAG chain

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### We will use two pre-defined chains to construct the final rag_chain:
In this exercise we are going to use two pre-defined chains to build the final chain:
* create_stuff_documents_chain
* create_retrieval_chain
* Let's learn a little bit more about these two pre-defined chains.

#### create_stuff_documents_chain
The create_stuff_documents_chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM. It passes ALL documents, so you should make sure it fits within the context window of the LLM you are using.
1. **Taking a List of Documents**: This function starts by receiving a group of documents that you provide.
  
2. **Formatting into a Prompt**: It then takes all these documents and organizes them into a specific prompt. A prompt is essentially a text setup that is used to feed information into a language model (like an LLM, or Large Language Model).

3. **Passing to an LLM**: After formatting the documents into a prompt, this function sends the formatted prompt to a language model. The model will process this information to perform tasks like answering questions, generating text, etc.

4. **Fit within Context Window**: The function sends all the documents at once to the LLM. However, it's important to make sure that the total length of the prompt does not exceed what the LLM can handle at one time. This limit is known as the "context window" of the LLM. If the prompt is too long, the model might not process it effectively.

In simpler terms, think of this chain as a way of taking several pieces of text, bundling them together in a specific way, and then feeding them to an LLM that reads and uses this bundled text to do its job. Just make sure the bundle isn’t too big for the LLM to handle at once!


#### create_retrieval_chain
The create_retrieval_chain takes in a user inquiry, which is then passed to the retriever to fetch relevant documents. Those documents (and original inputs) are then passed to an LLM to generate a response.
1. **Receiving a User Inquiry**: This process begins when a user asks a question or makes a request.

2. **Using a Retriever to Fetch Documents**: The function then uses a retriever to find documents that are relevant to the user's inquiry. This means it searches through available information to pick out parts that can help answer the question.

3. **Passing Information to an LLM**: After gathering the relevant documents, both these documents and the original user inquiry are sent to an LLM.

4. **Generating a Response**: The LLM processes all the information it receives to come up with an appropriate response, which is then given back to the user.

In simpler terms, this chain acts like a smart assistant that first looks up information based on your question, gathers useful details, and then uses those details along with your original question to craft a helpful answer.

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """You are an assistant for question-answering tasks.
    Use th following pieces of retrieved context to answer the question.
    If you don´t know the answer, say that you don´t know. Use three sentences maximum and
    keep the answer concise.\n\n
    {context}"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        ('human', '{input}')
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({'input': 'what is this article about'})

results['answer']

'The essay by Paul Graham discusses the importance of creating something that people want and not worrying too much about the business model initially. Graham also talks about the value of launching quickly, the concept of the "tamagotchi effect," and how helping users can lead to valuable information and confidence for startups. He also mentions examples like Craigslist and Google to illustrate his points.'

In [19]:
print(results['context'][0].metadata)

{'page': 0, 'source': './data/Be_Good.pdf'}
